In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Lab 3 Assignment
- In the exercises, you will still use the same dataset after One-Hot Encoding
- The dependent variable is "default payment next month"

__Name:__

## Starting point - Import and Split dataset
- Split the dataset into training and testing set, with a ratio of 9:1

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_curve

In [ ]:
df = pd.read_excel('http://fengmai.net//download/courses/2020S-BIA652NT-Pub/Week10-Classification%20and%20Hyperparameter%20Search%20using%20Python/default%20of%20credit%20card%20clients.xls', header = 1).drop('ID', axis = 1)

col = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
df2 = pd.get_dummies(df, columns = col, drop_first=True)
df2.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,PAY_0_-1,PAY_0_0,PAY_0_1,PAY_0_2,PAY_0_3,PAY_0_4,PAY_0_5,PAY_0_6,PAY_0_7,PAY_0_8,PAY_2_-1,PAY_2_0,PAY_2_1,PAY_2_2,PAY_2_3,...,PAY_2_7,PAY_2_8,PAY_3_-1,PAY_3_0,PAY_3_1,PAY_3_2,PAY_3_3,PAY_3_4,PAY_3_5,PAY_3_6,PAY_3_7,PAY_3_8,PAY_4_-1,PAY_4_0,PAY_4_1,PAY_4_2,PAY_4_3,PAY_4_4,PAY_4_5,PAY_4_6,PAY_4_7,PAY_4_8,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_5_4,PAY_5_5,PAY_5_6,PAY_5_7,PAY_5_8,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7,PAY_6_8
0,20000,24,3913,3102,689,0,0,0,0,689,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,120000,26,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,90000,34,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,50000,37,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,50000,57,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
y = df2['default payment next month'].values.astype(float)
x = df2.drop('default payment next month', axis = 1).values.astype(float)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

## Q1 - Grid-search both C and number of PCs 

Next, we use the gride search procedure to search for both the optimal C (inverse of L2 regularization parameter) and the number of principle components using cross validation on the train set.

- Define a parameter grid such that pca__n_components is 5 to 20, and add C as a parameter that can take the following values: [1000, 100, 10, 1, 0.1, 0.01]. Hint: use `pipe.get_params().keys()` to get the right key for C. 
- Which C and PC combination is the best?

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardization', 'pca', 'lg', 'standardization__copy', 'standardization__with_mean', 'standardization__with_std', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'lg__C', 'lg__class_weight', 'lg__dual', 'lg__fit_intercept', 'lg__intercept_scaling', 'lg__l1_ratio', 'lg__max_iter', 'lg__multi_class', 'lg__n_jobs', 'lg__penalty', 'lg__random_state', 'lg__solver', 'lg__tol', 'lg__verbose', 'lg__warm_start'])

In [ ]:
##### YOUR CODE HERE ######

pipe = Pipeline([
    ('standardization', preprocessing.StandardScaler()),
    ('pca', PCA()),    
    ('lg', LogisticRegression(solver = 'lbfgs', max_iter = 1000))
])

param_grid = {'pca__n_components':  range(5, 21),'lg__C':(1000, 100, 10, 1, 0.1, 0.01)}


grid = GridSearchCV(pipe, cv = 3, param_grid = param_grid, scoring = 'roc_auc', refit=True, verbose=1)
grid_fit = grid.fit(x_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed:  1.2min finished


In [ ]:
report(grid.cv_results_)

Model with rank: 1
Mean validation score: 0.762 (std: 0.005)
Parameters: {'lg__C': 100, 'pca__n_components': 18}

Model with rank: 2
Mean validation score: 0.761 (std: 0.005)
Parameters: {'lg__C': 1000, 'pca__n_components': 18}

Model with rank: 3
Mean validation score: 0.761 (std: 0.005)
Parameters: {'lg__C': 1000, 'pca__n_components': 20}



## Q2 - Provide AUC in the test set
- Based on Q1, use the best chosen model to provide AUC for the test set

In [ ]:
#### YOUR CODE HERE ####
grid_proba = grid.predict_proba(x_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, grid.predict_proba(x_test)[:,1])
print('AUC: {:.2f}'.format(auc(fpr, tpr)))

AUC: 0.75


## Q3 - Confusion matrix, precision, recall, F-1
- When decision threshold is 0.5:
    - Provide Confusion matrix, precision, recall, F-1 for the test set

In [ ]:
pred = grid.predict(x_test)

In [ ]:
pred

array([1., 0., 0., ..., 0., 0., 0.])

In [ ]:
# default thresold is 0.5 
from sklearn import metrics
metrics.confusion_matrix(y_test, pred)

array([[2257,  103],
       [ 478,  162]])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print('classification report:\n', classification_report(y_test, grid_proba[:,1] == 0.5))

classification report:
               precision    recall  f1-score   support

         0.0       0.79      1.00      0.88      2360
         1.0       0.00      0.00      0.00       640

    accuracy                           0.79      3000
   macro avg       0.39      0.50      0.44      3000
weighted avg       0.62      0.79      0.69      3000



## Q4 - Optimal decision threshold

If the average cost for a false positive prediction is 1, and the cost for a false negative prediction is 2, what is the optimal decision threshold that can minimize cost in the test set given your predicted_prob? Provide answer up to 2 decimal places. 

In [ ]:
y_pred = lg.predict

In [ ]:
probas = grid_proba[:,1]
probas

array([0.74129188, 0.23874296, 0.1857832 , ..., 0.17743378, 0.17770075,
       0.13056649])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
dt_cost = dict()

for dt in np.arange(0.0, 1.0, 0.01):
  # print(f"decision threshold: {dt:.2f}")
  #### YOUR CODE HERE ####
  des = (probas >= dt).astype(int)
  score = f1_score(y_test,des)
  dt_cost[dt] = score

In [ ]:
# dt_cost
min_cost = min(dt_cost.values())
ther = 0
for a, b in dt_cost.items():
   if(b == min_cost):
     ther = a


In [ ]:
print('Threshold=%.2f, F-Score=%.2f' % (ther, min_cost))

Threshold=0.99, F-Score=0.00


In [ ]:
dt_cost

{0.0: 0.3516483516483517,
 0.01: 0.35232590145884946,
 0.02: 0.3529087400055142,
 0.03: 0.3536247924737133,
 0.04: 0.355889724310777,
 0.05: 0.35712286593898684,
 0.06: 0.35734383792909397,
 0.07: 0.35914893617021276,
 0.08: 0.3640023001725129,
 0.09: 0.3676212741087084,
 0.1: 0.37196765498652296,
 0.11: 0.3817787418655098,
 0.12: 0.39341021416803956,
 0.13: 0.4077464788732394,
 0.14: 0.4217374665135859,
 0.15: 0.44319167389418906,
 0.16: 0.45999041686631537,
 0.17: 0.4731958762886597,
 0.18: 0.476029328821207,
 0.19: 0.4839296543359612,
 0.2: 0.4897435897435898,
 0.21: 0.49287169042769857,
 0.22: 0.49539985845718326,
 0.23: 0.5022222222222222,
 0.24: 0.5015290519877675,
 0.25: 0.5011820330969268,
 0.26: 0.49837662337662336,
 0.27: 0.49003322259136217,
 0.28: 0.4848993288590604,
 0.29: 0.4812286689419795,
 0.3: 0.47577854671280273,
 0.31: 0.4735915492957746,
 0.32: 0.46348061316501354,
 0.33: 0.46,
 0.34: 0.4523809523809524,
 0.35000000000000003: 0.4485981308411215,
 0.36: 0.4438095238

In [ ]:
# dt_cost = dict()

# for dt in np.arange(0.0, 1.0, 0.01):
#     # print(f"decision threshold: {dt:.2f}")
#     #### YOUR CODE HERE ####
#     des = (lg.predict_proba(x_test)[:,1] == dt).astype(int)
#     # preds = np.where(lg.predict_proba(x_test_s)[:,1] == dt, 1, 0)
    
#     # lg = LogisticRegression(class_weight = dt)
#     # lg.fit(x_train_s, y_train)
#     # prd = lg.predict(x_test_s)
#     # conf = metrics.confusion_matrix(y_test, prd)
#     # tn, fp, fn, tp = conf.ravel()
#     # cost = fp + fn
#     # dt_cost[dt] = cost
#     # print(f'tn: {tn}, fp: {fp}, fn: {fn}, tp: {tp}')
#     # print(conf)